## Project Code
Group Member: Layton Liu, Tommy Kong, Gary Lin

Objective: In this project, we will analyze the aviation accident over 1919-2023 and the causing factors behind.

Source: https://www.kaggle.com/datasets/warcoder/civil-aviation-accidents

In [1]:
import pandas as pd
import re

Since the data contains some incomprehensible value, we need to eliminate them first.

In [2]:
naValues = ['date unk.', 'unknown', 'Unknown country', '?']
data = pd.read_csv('aviation-accident.csv', na_values=naValues)
print(f'Original shape: {data.shape}')
data.dropna(subset=['date', 'fatalities', 'country'], axis=0, inplace=True)
data.shape

Original shape: (23917, 9)


(19825, 9)

The values in "fatalities" and "categories" are hard to read, thus we developed two functions to help us handle these values. We also developed an extra function to get the "season" when the accident happened based on the "date". These three functions are shown as follows:

In [3]:
def toInt(strValue):
    try:
        intValue = int(strValue)
        return intValue
    except Exception:
        return sum(map(int, re.findall('\d+', strValue)))

In [4]:
def catDetails(strValue):
    accidentCat = {'A': 'Accident', 'I': 'Incident', 'H': 'Hijacking', 'C': 'Criminal Occurrence', 'O': 'Other Occurrence', 'U': 'Unknown', '1': '(hull-loss)', '2': '(repairable damage)'}
    convertedValue = ''

    for char in strValue:
        convertedValue += accidentCat.get(char) + ' '

    return convertedValue.strip()


In [5]:
def getSeason(date):
    month = date.month

    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

Before analyzing the data, we organized it to increase readablity.

In [6]:
data['year'] = data['year'].astype('int64')
data['date'] = pd.to_datetime(data['date'], format='%d-%b-%Y', errors='coerce')
data['fatalities'] = data['fatalities'].map(toInt)
data = data.rename(columns={'cat':'category'})
data['category'] = data['category'].map(catDetails)
data['season'] = data['date'].map(getSeason)

In [7]:
data.dtypes

date            datetime64[ns]
type                    object
registration            object
operator                object
fatalities               int64
location                object
country                 object
category                object
year                     int64
season                  object
dtype: object

In [8]:
data = data.set_index(['year'])
data.head()

,date,type,registration,operator,fatalities,location,country,category,season
year,,,,,,,,,
1919,1919-08-02,Caproni Ca.48,NaN,Caproni,14,Verona,Italy,Accident (hull-loss),Summer
1919,1919-08-11,Felixstowe Fury,N123,RAF,1,near Felixtowe RNAS,U.K.,Accident (hull-loss),Summer
1920,1920-02-23,Handley Page O/7,G-EANV,Handley Page Transport,0,"Acadia Siding, C...",South Africa,Accident (hull-loss),Winter
1920,1920-02-25,Handley Page O/400,G-EAMC,Handley Page Transport,0,near El Shereik,Sudan,Accident (hull-loss),Winter
1920,1920-06-30,Handley Page O/400,G-EAKE,Handley Page Transport,0,ÃstanÃ¥,Sweden,Accident (hull-loss),Summer
